# Convergence of the exact strength function

This notebook presents the **convergence of the numerical evaluation of the exact strength function (SF)**, noted $S$:

$S(k) = - \frac{1}{\pi} \Im \left\langle g | G(k) | g \right\rangle$

where $G$ is the **Green's function** (or resolvent) of the Hamiltonian and $g$ is a test function.

The exact Green's function may be written as follows:

$G(k) = \sum_b \frac{\left| \varphi_b \right\rangle \left\langle \varphi_b \right|}{k^2/2 - {k_b}^2/2}
      + \sum_{p = \pm} \int_0^{+\infty} \text{d} k_1 \frac{\left| \varphi_p \right\rangle \left\langle \varphi_p \right|}{k^2/2 - {k_1}^2/2}$
      
where the first sum runs over the bound states (subscript $b$) (of wavenumber $k_b$), while the second one runs over the continuum states of wavenumber ($k_1$), which can be even (subscript $+$) or odd (subscript $-$).

It is studied in the case of the **1D Square-Well potential**, where the analytical continuum wavefunctions are known. The strength function can therefore be computed numerically. The numerical convergence of this exact computation has to be considered, and this is the purpose of the present notebook.

**Two numerical schemes to compute the the exact strength function will be presented:**
* the first one uses a ***large basis set of continuum states*** (of wavenumber $k_1$) evenly distributed (with a grid-step `hk`) on a given range of wavenumbers (from 0 to `kmax`). For each wavenumber $k$ where the strength function is computed, an integral over all the continuum states in the basis set is performed.
* the second one uses the fact that ***the integrals over the continuum can be made on a very limited range of wavenumbers rather than on the whole basis set***, given that the integrand is peaked around the wavenumber $k$ of the strength function grid. Therefore, defining a minimal continuum basis-set for each evaluation of the strength function allows drastic savings in computation time with a constrained numerical error, that allows to reach smaller `hk` values.

Also, to avoid the pole at $k = k_1$ in the integral over the continuum, one needs to add a infinitesimal number $i \eta$ in the denominator of the integrand. Therefore, the convergence of the two schemes with the decrease of this parameter `eta` must be discussed as well.

#### Import some modules and classes

In [ ]:
# Make the notebook aware of some of the SiegPy module classes
from siegpy import SWPBasisSet, Rectangular, Gaussian, SWPotential
# Other imports
import numpy as np
import matplotlib.pyplot as plt

## Define a 1D Square-Well Potential.

A 1D SW potential is caracterized by its width *l* and depth *V0*.

In [ ]:
V0 = 10  # depth of the potential
l = np.sqrt(2) * np.pi  # width of the potential
potential = SWPotential(l, V0)

## Create a basis-set made of Siegert states only.

Only to allow the creation of an exact basis set and to be able to compare the exact SF with the MLE of the SF.

In [ ]:
re_kmax = 20.
im_kmax = 3.
re_hk = 1.
im_hk = 2.
siegerts = SWPBasisSet.find_Siegert_states(potential, re_kmax, re_hk, im_kmax, im_hk=im_hk, analytic=True)

## Create an exact basis-set, made of bound and continuum states.

The bound states can be taken from the previous calculation finding the Siegert states (given that bound states are nothing but a particuliar type of Siegert states), while the continuum states are discretized over a grid of real wavenumbers. This grid is caracterized by the grid step `hk` and the maximal wavenumber `kmax`.

In [ ]:
# Find some continuum states
hk = 0.01
kmax = 10
continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)

# Create the exact basis set
exact = siegerts.bounds + continuum

## Define a test function to evaluate the strength function.

The strength function (SF) is evaluated using a test function $g$. For the MLE of the SF to hold, the test function must lie in region $II$ (inside the potential, where $|x| \leq l/2$). The cases of a Gaussian test function and of a rectangular test function are studied.

In [ ]:
xc = 0.0  # center of the test functions
a = l/2.  # width of the rectangular function
sigma = l/20.  # width of the Gaussian
gauss = Gaussian(sigma, xc)  # Gaussian test function
rect = Rectangular.from_width_and_center(a, xc)  # Rectangular test function

## Create the grid of wavenumbers where the strength function is evaluated

It is not mandatory to keep the same grid as for the continuum states, and it is strongly advised to decrease this number as much as possible to save computation time, since an integration over all continuum states is performed for each point in this grid, that only defines the grid used for plotting.

In [ ]:
hk = 0.1  # Grid-step for plotting
kmax = 10  # Maximal wavenumber for plotting
kgrid = np.arange(hk, kmax, hk)

## Strength function for a Gaussian test function

Both exact and approximate strength functions are calculated, the latter being very cheap to obtain, whereas the exact SF takes a long time due to the integrations over the continuum, that must extend to a large `kmax` and be very dense (small `hk`). It is very difficult to get a converged result with respect to the amelioration of the continuum states basis (*i.e.* with smaller `hk` and larger `kmax`). Learning from the numerical convergence of this scheme, it was possible to produce another scheme allowing a better convergence (using smaller `eta` and `hk`) within a reasonable amount of time, by avoiding the need to discretize a dense basis set of continuum states over the whole range of wavenumbers.

Note that there is no need to use a basis made of continuum states to compute the exact strength function "on the fly". Some default values for `hk`, `eta` and a tolerance parameters are passed to this method. They were found to give a good compromise between accuracy and speed for a large number of cases.

Both MLE and converged exact SF give very comparable results.

In [ ]:
SF_MLE_gauss         = siegerts.MLE_strength_function(gauss, kgrid)
SF_exact_gauss       = exact.exact_strength_function(gauss, kgrid)
SF_exact_gauss_other = siegerts.exact_strength_function_OTF(gauss, kgrid)

In [ ]:
# Compare the exact SF to the MLE of the SF
plt.plot(kgrid, SF_MLE_gauss, color='k', lw=5, label='MLE')
plt.plot(kgrid, SF_exact_gauss, label='Exact')
plt.plot(kgrid, SF_exact_gauss_other, label='Exact (converged)')
plt.xlabel("$k$")
plt.ylabel("$S(k)$")
plt.title('Gaussian test function')
plt.legend()
plt.show()

## Tests of convergence of the exact strength function using the whole continuum basis set

Some parameters are important to get converged exact strength functions:
* `hk` is the wavenumber step between two adjacent continuum states in the basis set. The smaller it gets, the denser the basis-set is.
* `kmax` is the highest wavenumber of a continuum state in the basis set. The larger it is, the more complete the basis set is.
* `eta` is a the imaginary part of a number added to the denominator of the integrand in the definition of the Green's function to avoid poles alomg the real axis. It must tend to 0.

### Convergence with respect to `kmax`

Increasing `kmax` allows for the discretization of the strength function to reach higher wavenumbers.

The integrand for each wavenumber in the grid is peaked around the value of the wavenumber, so it might be quicker (and allowing greater precision) to only integrate around each wavenumber of the plotting grid, rather than all the continuum grid. Therefore, there is still a need for a rather dense basis set of continuum states, but only around each point of the desired plotting grid.

In [ ]:
hk = 0.01
results_hk = {}
for kmax in [1, 3, 5, 7.5, 10]:
    print("kmax =", kmax)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_hk[str(kmax)] = exact.exact_strength_function(gauss, kgrid)

In [ ]:
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_hk.items():
    plt.plot(kgrid, v, label='$k_{max}$ = '+k)
plt.legend()
plt.show()

### Convergence with respect to `hk`

There is a convergence as long as `hk` decreases, but it really depends on the `eta` value. 
For a given `eta`, a convergence is reached approximately when it is five times greater than `hk`.

#### `eta` = 0.1

In [ ]:
kmax = 10.
results_kmax_1 = {}
for hk in [0.1, 0.05, 0.01, 0.005]:
    print("hk =", hk)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_kmax_1[str(hk)] = exact.exact_strength_function(gauss, kgrid, eta=0.1)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_kmax_1.items():
    plt.plot(kgrid, v, label='$h_k$ = '+k)
plt.legend()
plt.show()

In [ ]:
for k, v in results_kmax_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$h_k$ = '+k)
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_kmax_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$h_k$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

#### `eta` = 0.05

The convergence is reached for `hk` = 0.01

In [ ]:
kmax = 10.
results_kmax_2 = {}
for hk in [0.1, 0.05, 0.01, 0.005]:
    print("hk =", hk)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_kmax_2[str(hk)] = exact.exact_strength_function(gauss, kgrid, eta=0.05)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_kmax_2.items():
    plt.plot(kgrid, v, label='$h_k$ = '+k)
plt.legend()
plt.show()

In [ ]:
for k, v in results_kmax_2.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$h_k$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_kmax_2.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$h_k$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

#### *eta* = 0.01

In [ ]:
kmax = 10.
results_kmax_3 = {}
for hk in [0.1, 0.05, 0.01, 0.005, 0.002]:
    print("hk =", hk)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_kmax_3[str(hk)] = exact.exact_strength_function(gauss, kgrid, eta=0.01)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_kmax_3.items():
    plt.plot(kgrid, v, label='$h_k$ = '+k)
plt.legend()
plt.show()

In [ ]:
for k, v in results_kmax_3.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$h_k$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_kmax_3.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$h_k$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

### Convergence with respect to `eta`

For a given `hk`, there is a value of `eta` that maximizes the closeness to the MLE of the SF. This parameter must be chosen carefully to get high quality results of the computation. 

It seems that the best choice is $\frac{eta}{h_k} \in [5,10]$ for reasonably small `hk` values. The default value of eta is therefore equal to 5 `hk`.

#### `hk` = 0.01

In [ ]:
kmax = 10.
hk = 0.01
results_eta_1 = {}
continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
exact = siegerts.bounds + continuum
for eta in [1.0, 0.5, 0.1, 0.06, 0.05, 0.04, 0.025, 0.01, 0.001]:
    print("eta =", eta)
    # print("Number of states in the basis-set:", len(exact))
    results_eta_1[str(eta)] = exact.exact_strength_function(gauss, kgrid, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_eta_1.items():
    plt.plot(kgrid, v, label='$\eta$ = '+k)
plt.ylim(0, 0.0062)
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_eta_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$\eta$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_eta_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$\eta$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

#### `hk` = 0.002

In [ ]:
kmax = 10.
hk = 0.002
results_eta_2 = {}
continuum = SWPBasisSet.find_continuum_states(potential, kmax, hk)
exact = siegerts.bounds + continuum
for eta in [0.5, 0.05, 0.03, 0.02, 0.01, 0.00875, 0.0075, 0.001, 0.0005]:
    print("eta =", eta)
    # print("Number of states in the basis-set:", len(exact))
    results_eta_2[str(eta)] = exact.exact_strength_function(gauss, kgrid, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_eta_2.items():
    plt.plot(kgrid, v, label='$\eta$ = '+k)
plt.ylim(0, 0.0062)
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_eta_2.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$\eta$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_eta_2.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$\eta$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

### Convergence by getting `hk` and `eta` smaller at the same time

There is a systematic improvement as both `hk` and `eta` decrase: the exact SF converges to the MLE of the SF (taken as the reference, even though it should be the other way around, but the quality of the MLE of the SF is so good that it is easy to take it as a reference).

The ratio `eta` / `hk` = 10 seems to be preferable.

#### `eta` / `hk` = 10

In [ ]:
kmax = 10.
hk = 0.01
eta = 0.1
results_factor_1 = {}
for factor in [1, 2, 5, 10, 20]:
    new_eta = eta / factor
    new_hk = hk / factor
    print("eta =", new_eta, " ; hk =", new_hk)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, new_hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_factor_1[str(factor)] = exact.exact_strength_function(gauss, kgrid, eta=new_eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_factor_1.items():
    plt.plot(kgrid, v, label='factor = '+k)
plt.plot()
plt.legend()
plt.show()

In [ ]:
for k, v in results_factor_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='factor = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_factor_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='factor = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

#### `eta` / `hk` = 5

In [ ]:
kmax = 10.
hk = 0.01
eta = 0.5
results_factor_2 = {}
for factor in [1, 2, 5, 10, 20]:
    new_eta = eta / factor
    new_hk = hk / factor
    print("eta =", new_eta, " ; hk =", new_hk)
    continuum = SWPBasisSet.find_continuum_states(potential, kmax, new_hk)
    exact = siegerts.bounds + continuum
    # print("Number of states in the basis-set:", len(exact))
    results_factor_2[str(factor)] = exact.exact_strength_function(gauss, kgrid, eta=new_eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in results_factor_2.items():
    plt.plot(kgrid, v, label='factor = '+k)
plt.plot()
plt.legend()
plt.show()

In [ ]:
for k, v in results_factor_2.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='factor = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in results_factor_2.items():
    plt.plot(kgrid, abs(v-MLE_g), label='factor = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

## Test of the other scheme to get the exact strength function

As experienced above, it is difficult to get an exact strength function converged with respect to the basis set of continuum states. This is due to the fact that, the denser the basis set, the longer it takes to perform the integral over the continuum. This integral being performed for each wavenumber where the strength function is evaluated, it can rapidly become overwhelmimgly long.

However, each integral is peaked around the wavenumber where the strength function is evaluated and vanishes quickly (this can be inferred from the tests performed with respect to the basis set range `kmax`). We take advantage of that fact in this new method for computing the exact strength function calculation, so that is takes a (way) shorter time to get a converged strength function. The idea is to perform the minimal integrals over the continuum, given some input parameters:
* `hk` is the spacing between two consecutive continuum states used to build the integrand for each point in the wavenumber grid.
* `tol` is a tolerance value: a new point is appended to the integrand function if the ratio between the new value of the integrand and the maximum of the integrand is superior to that tolerance. 
* `eta` is the same infinitesimal number as in the other scheme (used to avoid the infinite value of the integrand for each wavenumber of the grid)

The first example shows the exact SF using the default values for the "on the fly" exact evaluation of the SF for a Gaussian test function. 

In [ ]:
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
sf_g = siegerts.exact_strength_function_OTF(gauss, kgrid)
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
plt.plot(kgrid, sf_g, label="exact")
plt.legend()
plt.show()

The second example is the same as the previous one, but for a rectangular test function. 

In [ ]:
MLE_r = siegerts.MLE_strength_function(rect, kgrid)
sf_r = siegerts.exact_strength_function_OTF(rect, kgrid)
plt.plot(kgrid, MLE_r, label="MLE", lw=5, color='k')
plt.plot(kgrid, sf_r, label="exact")
plt.legend()
plt.show()

Here are the results obtained when varying the default values of `hk` and `tol` for a Gaussian test function.

In [ ]:
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
sf_1 = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=0.0001, tol=0.0001, eta=0.001) #default
sf_2 = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=0.0001, tol=0.00001, eta=0.001)
sf_3 = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=0.001, tol=0.00001, eta=0.001)
sf_4 = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=0.00001, tol=0.00001, eta=0.001)
sf_5 = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=0.00001, tol=0.0001, eta=0.001)

In [ ]:
for sf in sf_1, sf_2, sf_3, sf_4, sf_5:
    plt.plot(kgrid, abs(sf-MLE_g))
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.show()

In [ ]:
for sf in sf_1, sf_2, sf_3, sf_4, sf_5:
    plt.plot(kgrid, abs(sf-MLE_g)/MLE_g)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.show()

### Convergence with respect to `hk`

There is a convergence as `hk` decreases for a given `eta`. Reach smaller values of `hk` is not prohibitive (the default value is already smaller than the ones used with the other scheme)

#### `eta` = 10$^{-3}$

In [ ]:
tol = 10**(-5)
eta = 10**(-3)
result_hk_1 = {}
for hk in [10**(-n) for n in [2, 3, 4]]:
    print(hk)
    result_hk_1[str(hk)] = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=hk, tol=tol, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in result_hk_1.items():
    plt.plot(kgrid, v, label='$h_k$ = '+k)
plt.plot()
plt.ylim(0.0, 0.008)
plt.legend()
plt.show()

In [ ]:
for k, v in result_hk_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$h_k$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_hk_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$h_k$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

#### `eta` = 10$^{-4}$

In [ ]:
eta = 10**(-4)
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
result_hk_2 = {}
for hk in [10**(-n) for n in [2, 3, 4, 5]]:
    print(hk)
    result_hk_2[str(hk)] = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=hk, tol=tol, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in result_hk_2.items():
    plt.plot(kgrid, v, label='$h_k$ = '+k)
plt.plot()
plt.ylim(0.0, 0.008)
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_hk_2.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='$h_k$ = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_hk_2.items():
    plt.plot(kgrid, abs(v-MLE_g), label='$h_k$ = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

### Convergence with respect to `tol`

The results converge as tol is decreased, for a given set of `hk` and `eta`. At some point, it is not efficient to decrease the tolerence to get more converged results (the gain in precision being rather small for the extra cost).

In [ ]:
hk = 10**(-4)
eta = 10**(-3)
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
result_tol_1 = {}
for tol in [10**(-n) for n in [2, 3, 4, 5, 6, 7]]:
    print(tol)
    result_tol_1[str(tol)] = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=hk, tol=tol, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in result_tol_1.items():
    plt.plot(kgrid, v, label='tol = '+k)
plt.plot()
plt.ylim(0.0, 0.008)
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_tol_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='tol = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_tol_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='tol = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

### Convergence with respect to `eta`

For a given `hk`, there is an optimal `eta`, which is the same as the one experienced using the other scheme: around 10 times the value of `hk`.

In [ ]:
hk = 10**(-4)
tol = 10**(-4)
MLE_g = siegerts.MLE_strength_function(gauss, kgrid)
result_eta_1 = {}
for eta in [10**(-n) for n in [2, 3, 4, 5, 6]]:
    print(eta)
    result_eta_1[str(eta)] = siegerts.exact_strength_function_OTF(gauss, kgrid, hk=hk, tol=tol, eta=eta)

In [ ]:
plt.plot(kgrid, MLE_g, label="MLE", lw=5, color='k')
for k, v in result_eta_1.items():
    plt.plot(kgrid, v, label='eta = '+k)
plt.plot()
plt.ylim(0.0, 0.008)
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_eta_1.items():
    plt.plot(kgrid, abs(v-MLE_g)/MLE_g, label='eta = '+k)
plt.plot()
plt.title('Relative error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
for k, v in result_eta_1.items():
    plt.plot(kgrid, abs(v-MLE_g), label='eta = '+k)
plt.plot()
plt.title('Absolute Error')
plt.yscale('log')
plt.legend(loc=6, bbox_to_anchor=(1, 0.5))
plt.show()